In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

## Creación de datos temporales con múltiples pacientes

In [ ]:
np.random.seed(42)

# Generar datos para 5 pacientes durante 3 días
pacientes = ['P001', 'P002', 'P003', 'P004', 'P005']
inicio = pd.Timestamp('2025-01-01 00:00:00')
n_horas = 72  # 3 días

datos = []
for paciente in pacientes:
    for hora in range(n_horas):
        timestamp = inicio + timedelta(hours=hora)
        
        # Simular variación circadiana
        variacion_hora = np.sin(hora * np.pi / 12) * 5
        
        datos.append({
            'paciente_id': paciente,
            'timestamp': timestamp,
            'HR': 75 + variacion_hora + np.random.normal(0, 8),
            'SBP': 120 + variacion_hora + np.random.normal(0, 10),
            'DBP': 80 + variacion_hora/2 + np.random.normal(0, 5),
            'SpO2': 97 + np.random.normal(0, 1.5),
            'temperatura': 37.0 + np.random.normal(0, 0.3)
        })

df = pd.DataFrame(datos)

# Añadir información derivada
df['fecha'] = df['timestamp'].dt.date
df['hora'] = df['timestamp'].dt.hour
df['dia_semana'] = df['timestamp'].dt.day_name()

# Clasificar por turno
def asignar_turno(hora):
    if 7 <= hora < 15:
        return 'Mañana'
    elif 15 <= hora < 23:
        return 'Tarde'
    else:
        return 'Noche'

df['turno'] = df['hora'].apply(asignar_turno)

# Clasificar estado clínico
def clasificar_estado(row):
    if row['HR'] > 100 or row['SBP'] > 140 or row['SpO2'] < 94:
        return 'Crítico'
    elif row['HR'] > 90 or row['SBP'] > 130 or row['SpO2'] < 96:
        return 'Alerta'
    else:
        return 'Normal'

df['estado'] = df.apply(clasificar_estado, axis=1)

print("=== Dataset generado ===")
print(df.head(15))
print(f"\nTotal de registros: {len(df)}")
print(f"Pacientes: {df['paciente_id'].nunique()}")
print(f"Periodo: {df['timestamp'].min()} a {df['timestamp'].max()}")

## 1. Agrupación por paciente

In [ ]:
# Agregaciones básicas por paciente
print("=== Resumen por paciente ===")

resumen_paciente = df.groupby('paciente_id').agg({
    'HR': ['mean', 'min', 'max', 'std'],
    'SBP': ['mean', 'min', 'max', 'std'],
    'SpO2': ['mean', 'min', 'max'],
    'timestamp': 'count'
}).round(2)

resumen_paciente.columns = ['_'.join(col) for col in resumen_paciente.columns]
resumen_paciente.rename(columns={'timestamp_count': 'n_registros'}, inplace=True)

print(resumen_paciente)

## 2. Agrupación por hora del día

In [ ]:
# Análisis de patrones por hora del día
print("=== Patrones por hora del día ===")

resumen_hora = df.groupby('hora').agg({
    'HR': 'mean',
    'SBP': 'mean',
    'DBP': 'mean',
    'SpO2': 'mean',
    'temperatura': 'mean'
}).round(2)

print(resumen_hora)

# Identificar horas con valores más altos/bajos
print(f"\nHora con HR más alto: {resumen_hora['HR'].idxmax()}:00 ({resumen_hora['HR'].max():.1f} bpm)")
print(f"Hora con HR más bajo: {resumen_hora['HR'].idxmin()}:00 ({resumen_hora['HR'].min():.1f} bpm)")
print(f"Hora con SBP más alto: {resumen_hora['SBP'].idxmax()}:00 ({resumen_hora['SBP'].max():.1f} mmHg)")

## 3. Agrupación por turno

In [ ]:
# Análisis por turno de trabajo
print("=== Análisis por turno ===")

resumen_turno = df.groupby('turno').agg({
    'HR': ['mean', 'std'],
    'SBP': ['mean', 'std'],
    'SpO2': ['mean', 'std'],
    'paciente_id': 'count'
}).round(2)

resumen_turno.columns = ['_'.join(col) for col in resumen_turno.columns]
resumen_turno.rename(columns={'paciente_id_count': 'n_registros'}, inplace=True)

print(resumen_turno)

# Comparar variabilidad entre turnos
print("\n=== Variabilidad por turno ===")
print(f"Turno con mayor variabilidad en HR: {resumen_turno['HR_std'].idxmax()}")
print(f"Turno más estable: {resumen_turno['HR_std'].idxmin()}")

## 4. Agrupación por estado clínico

In [ ]:
# Análisis por estado clínico
print("=== Distribución de estados clínicos ===")

# Conteo de estados
conteo_estados = df['estado'].value_counts()
print("\nConteo:")
print(conteo_estados)
print("\nPorcentajes:")
print((conteo_estados / len(df) * 100).round(2))

# Características por estado
print("\n=== Signos vitales por estado ===")
resumen_estado = df.groupby('estado').agg({
    'HR': ['mean', 'std'],
    'SBP': ['mean', 'std'],
    'SpO2': ['mean', 'std']
}).round(2)

print(resumen_estado)

## 5. Agrupación múltiple: Paciente y turno

In [ ]:
# Análisis combinado: paciente y turno
print("=== Análisis por paciente y turno ===")

resumen_paciente_turno = df.groupby(['paciente_id', 'turno']).agg({
    'HR': 'mean',
    'SBP': 'mean',
    'SpO2': 'mean'
}).round(2)

print(resumen_paciente_turno)

## 6. Agregaciones personalizadas

In [ ]:
# Función personalizada para calcular rango
def rango(serie):
    return serie.max() - serie.min()

# Función para calcular coeficiente de variación
def coef_variacion(serie):
    return (serie.std() / serie.mean()) * 100

print("=== Agregaciones personalizadas por paciente ===")

resumen_personalizado = df.groupby('paciente_id').agg({
    'HR': ['mean', rango, coef_variacion],
    'SBP': ['mean', rango, coef_variacion]
}).round(2)

resumen_personalizado.columns = ['_'.join(col) for col in resumen_personalizado.columns]
print(resumen_personalizado)

## 7. Variabilidad diaria

In [ ]:
# Análisis de variabilidad día a día
print("=== Variabilidad diaria ===")

resumen_diario = df.groupby(['paciente_id', 'fecha']).agg({
    'HR': ['mean', 'min', 'max', 'std'],
    'SBP': ['mean', 'min', 'max', 'std'],
    'estado': lambda x: (x == 'Crítico').sum()  # Contar episodios críticos
}).round(2)

resumen_diario.columns = ['_'.join(col) if col[1] else col[0] for col in resumen_diario.columns]
resumen_diario.rename(columns={'estado_<lambda>': 'episodios_criticos'}, inplace=True)

print(resumen_diario.head(10))

## 8. Detección de episodios críticos

In [ ]:
# Identificar y contar episodios críticos por paciente
print("=== Episodios críticos por paciente ===")

episodios_criticos = df[df['estado'] == 'Crítico'].groupby('paciente_id').agg({
    'timestamp': 'count',
    'HR': ['mean', 'max'],
    'SBP': ['mean', 'max'],
    'SpO2': ['mean', 'min']
}).round(2)

episodios_criticos.columns = ['_'.join(col) for col in episodios_criticos.columns]
episodios_criticos.rename(columns={'timestamp_count': 'n_episodios'}, inplace=True)

print(episodios_criticos)

# Paciente con más episodios críticos
paciente_riesgo = episodios_criticos['n_episodios'].idxmax()
print(f"\n⚠️ Paciente con más episodios críticos: {paciente_riesgo} ({episodios_criticos.loc[paciente_riesgo, 'n_episodios']:.0f} episodios)")

## 9. Tabla dinámica (pivot table) avanzada

In [ ]:
# Tabla dinámica: HR promedio por paciente y turno
print("=== Tabla dinámica: HR promedio ===")

tabla_hr = pd.pivot_table(
    df,
    values='HR',
    index='paciente_id',
    columns='turno',
    aggfunc='mean',
    margins=True,  # Añade totales
    margins_name='Promedio'
).round(2)

print(tabla_hr)

# Tabla dinámica: Conteo de estados por paciente y turno
print("\n=== Tabla dinámica: Estados críticos por turno ===")

tabla_estados = pd.pivot_table(
    df[df['estado'] == 'Crítico'],
    values='HR',
    index='paciente_id',
    columns='turno',
    aggfunc='count',
    fill_value=0
)

print(tabla_estados)

## 10. Tabla resumen completa por paciente

In [ ]:
# Crear tabla resumen comprehensiva
print("=== TABLA RESUMEN COMPLETA POR PACIENTE ===")

tabla_resumen = df.groupby('paciente_id').agg({
    # Conteos
    'timestamp': 'count',
    
    # Heart Rate
    'HR': ['mean', 'min', 'max', 'std'],
    
    # Blood Pressure
    'SBP': ['mean', 'min', 'max'],
    'DBP': ['mean', 'min', 'max'],
    
    # SpO2
    'SpO2': ['mean', 'min'],
    
    # Estados
    'estado': [lambda x: (x == 'Crítico').sum(), 
               lambda x: (x == 'Alerta').sum(),
               lambda x: (x == 'Normal').sum()]
}).round(2)

# Renombrar columnas
tabla_resumen.columns = [
    'n_registros',
    'HR_mean', 'HR_min', 'HR_max', 'HR_std',
    'SBP_mean', 'SBP_min', 'SBP_max',
    'DBP_mean', 'DBP_min', 'DBP_max',
    'SpO2_mean', 'SpO2_min',
    'episodios_criticos', 'episodios_alerta', 'episodios_normales'
]

print(tabla_resumen)

# Calcular porcentaje de tiempo en estado crítico
tabla_resumen['%_critico'] = (tabla_resumen['episodios_criticos'] / tabla_resumen['n_registros'] * 100).round(2)
print("\n=== Porcentaje de tiempo en estado crítico ===")
print(tabla_resumen[['episodios_criticos', '%_critico']].sort_values('%_critico', ascending=False))

## 11. Tendencias temporales

In [ ]:
# Visualizar tendencias por hora del día
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Gráfico 1: HR promedio por hora
resumen_hora.plot(y='HR', ax=axes[0], marker='o', linewidth=2, color='blue')
axes[0].set_title('Heart Rate promedio por hora del día', fontweight='bold', fontsize=12)
axes[0].set_xlabel('Hora del día')
axes[0].set_ylabel('HR (bpm)')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=resumen_hora['HR'].mean(), color='red', linestyle='--', 
                label=f'Media general: {resumen_hora["HR"].mean():.1f}')
axes[0].legend()

# Gráfico 2: SBP promedio por hora
resumen_hora.plot(y='SBP', ax=axes[1], marker='s', linewidth=2, color='green')
axes[1].set_title('Presión Arterial Sistólica promedio por hora del día', fontweight='bold', fontsize=12)
axes[1].set_xlabel('Hora del día')
axes[1].set_ylabel('SBP (mmHg)')
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=resumen_hora['SBP'].mean(), color='red', linestyle='--', 
                label=f'Media general: {resumen_hora["SBP"].mean():.1f}')
axes[1].legend()

plt.tight_layout()
plt.show()

print("Gráficos de tendencias generados")

## 12. Comparación entre turnos (visualización)

In [ ]:
# Boxplot comparativo por turno
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# HR por turno
df.boxplot(column='HR', by='turno', ax=axes[0])
axes[0].set_title('Heart Rate por turno')
axes[0].set_xlabel('Turno')
axes[0].set_ylabel('HR (bpm)')

# SBP por turno
df.boxplot(column='SBP', by='turno', ax=axes[1])
axes[1].set_title('Presión Arterial Sistólica por turno')
axes[1].set_xlabel('Turno')
axes[1].set_ylabel('SBP (mmHg)')

# SpO2 por turno
df.boxplot(column='SpO2', by='turno', ax=axes[2])
axes[2].set_title('Saturación de Oxígeno por turno')
axes[2].set_xlabel('Turno')
axes[2].set_ylabel('SpO2 (%)')

plt.suptitle('')  # Eliminar título automático
plt.tight_layout()
plt.show()

print("Comparación por turnos generada")

## 13. Reporte ejecutivo por paciente

In [ ]:
# Generar reporte ejecutivo individual
def generar_reporte_paciente(paciente_id):
    datos_paciente = df[df['paciente_id'] == paciente_id]
    
    print(f"\n{'='*70}")
    print(f"REPORTE CLÍNICO - {paciente_id}")
    print(f"{'='*70}")
    
    print(f"\nPeriodo: {datos_paciente['timestamp'].min()} a {datos_paciente['timestamp'].max()}")
    print(f"Total de mediciones: {len(datos_paciente)}")
    
    print("\n--- SIGNOS VITALES PROMEDIO ---")
    print(f"HR: {datos_paciente['HR'].mean():.1f} ± {datos_paciente['HR'].std():.1f} bpm")
    print(f"SBP: {datos_paciente['SBP'].mean():.1f} ± {datos_paciente['SBP'].std():.1f} mmHg")
    print(f"DBP: {datos_paciente['DBP'].mean():.1f} ± {datos_paciente['DBP'].std():.1f} mmHg")
    print(f"SpO2: {datos_paciente['SpO2'].mean():.1f} ± {datos_paciente['SpO2'].std():.1f} %")
    
    print("\n--- DISTRIBUCIÓN DE ESTADOS ---")
    estados = datos_paciente['estado'].value_counts()
    for estado, conteo in estados.items():
        porcentaje = (conteo / len(datos_paciente)) * 100
        print(f"{estado}: {conteo} ({porcentaje:.1f}%)")
    
    print("\n--- ANÁLISIS POR TURNO ---")
    turno_stats = datos_paciente.groupby('turno')['HR'].agg(['mean', 'std']).round(1)
    for turno, row in turno_stats.iterrows():
        print(f"{turno}: HR = {row['mean']} ± {row['std']} bpm")
    
    # Identificar periodos críticos
    criticos = datos_paciente[datos_paciente['estado'] == 'Crítico']
    if len(criticos) > 0:
        print("\n⚠️ ALERTAS:")
        print(f"- {len(criticos)} episodios críticos detectados")
        print(f"- HR máximo: {datos_paciente['HR'].max():.1f} bpm")
        print(f"- SBP máximo: {datos_paciente['SBP'].max():.1f} mmHg")
        print(f"- SpO2 mínimo: {datos_paciente['SpO2'].min():.1f} %")
    else:
        print("\n✓ Sin episodios críticos")
    
    print(f"\n{'='*70}\n")

# Generar reportes para los primeros 2 pacientes
for paciente in ['P001', 'P002']:
    generar_reporte_paciente(paciente)

## Resumen

### Técnicas de agrupamiento:

1. **groupby()** simple: Agrupar por una variable
2. **groupby()** múltiple: Agrupar por varias variables simultáneamente
3. **agg()**: Aplicar múltiples funciones de agregación
4. **pivot_table()**: Crear tablas dinámicas con agregaciones

### Agregaciones comunes:
- **mean**: Media aritmética
- **min/max**: Valores extremos
- **std**: Desviación estándar (variabilidad)
- **count**: Conteo de registros
- **sum**: Suma total
- Funciones personalizadas con **lambda**

### Aplicaciones clínicas:

1. **Por paciente**: Perfil individual de signos vitales
2. **Por turno**: Identificar patrones de cuidado
3. **Por hora**: Detectar variaciones circadianas
4. **Por estado**: Caracterizar episodios críticos
5. **Temporal**: Tendencias diarias y semanales

### Interpretación clínica:
- **Variabilidad alta**: Puede indicar inestabilidad hemodinámica
- **Tendencias por turno**: Útil para evaluar calidad de cuidado
- **Episodios críticos**: Requieren análisis detallado de causas
- **Patrones horarios**: Importantes para ajustar tratamientos